In [2]:
import tensorflow as tf
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions, VGG16
import numpy as np


# Step 1: Explore the VGG16 model and its training data
# VGG16 model
model = VGG16(weights='imagenet')
model.summary()



2024-02-27 07:09:45.783631: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
# Step 3: Load images and predict their classes using the pre-trained model
def predict_image_class(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    predictions = model.predict(img_array)
    print('Predicted:', decode_predictions(predictions, top=3)[0])

In [14]:
# Load sample images and predict their classes
image_paths = ['MCL750s.jpg', 'mango.jpg', 'dog.jpg', 'pikachu.jpg']
for img_path in image_paths:
    predict_image_class(img_path, model)

1/1 [==============================] - 0s 211ms/step
Predicted: [('n04285008', 'sports_car', 0.9579621), ('n04037443', 'racer', 0.02794775), ('n03100240', 'convertible', 0.009576104)]
1/1 [==============================] - 0s 207ms/step
Predicted: [('n07749582', 'lemon', 0.78642094), ('n07747607', 'orange', 0.11885359), ('n07720875', 'bell_pepper', 0.018098714)]
1/1 [==============================] - 0s 204ms/step
Predicted: [('n02113712', 'miniature_poodle', 0.39879838), ('n02113799', 'standard_poodle', 0.3789748), ('n02113624', 'toy_poodle', 0.09292235)]
1/1 [==============================] - 0s 221ms/step
Predicted: [('n03476684', 'hair_slide', 0.6169893), ('n02951585', 'can_opener', 0.11346942), ('n03944341', 'pinwheel', 0.070913546)]


In [6]:
import keras
model_without_last_layer = keras.Model(inputs=model.input, outputs=model.layers[-2].output)
model_without_last_layer.summary()

# For example, let's add a couple of dense layers
new_model = keras.Sequential([
    model_without_last_layer,
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])
new_model.summary()

# Assuming you have data prepared in X_train, y_train, X_test, y_test
# Compile the model
new_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# # Train the model
# history = new_model.fit(X_train, y_train,
#                         epochs=10,
#                         batch_size=32,
#                         validation_data=(X_test, y_test))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
# Load sample images and predict their classes
image_paths = ['MCL750s.jpg', 'mango.jpg']
for img_path in image_paths:
    predict_image_class(img_path, model)

1/1 [==============================] - 0s 213ms/step
Predicted: [('n04285008', 'sports_car', 0.9579621), ('n04037443', 'racer', 0.02794775), ('n03100240', 'convertible', 0.009576104)]
1/1 [==============================] - 0s 206ms/step
Predicted: [('n07749582', 'lemon', 0.78642094), ('n07747607', 'orange', 0.11885359), ('n07720875', 'bell_pepper', 0.018098714)]


In [20]:
import tensorflow as tf
from keras.datasets import mnist
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, UpSampling2D
from keras.optimizers import Adam
import numpy as np

# Step 1: Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0

# Convert labels to binary: 0 if not digit 5, 1 if digit 5
y_train_binary = (y_train == 5).astype(int)
y_test_binary = (y_test == 5).astype(int)

# Step 4: Remove the last layer of the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Step 5: Add layers to the VGG16 model
x_model = Sequential([
    UpSampling2D(size=(8, 8), input_shape=(28, 28, 1)),  # Upsample to match VGG input size
    new_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Step 7: Train the new model for binary classification
# Compile the model
x_model.compile(optimizer=Adam(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
x_model.fit(X_train, y_train_binary,
          epochs=5,
          batch_size=32,
          validation_data=(X_test, y_test_binary))

ValueError: Exception encountered when calling layer 'model' (type Functional).

Input 0 of layer "block1_conv1" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (None, 224, 224, 1)

Call arguments received by layer 'model' (type Functional):
  • inputs=tf.Tensor(shape=(None, 224, 224, 1), dtype=float32)
  • training=None
  • mask=None